In [1]:
from stackapi import StackAPI
import os
import json
import time
import pandas as pd
import numpy as np

# 1 - Preprocess meanings

In [2]:
path = '../Dati wiki/pulito_nodes.csv'

In [3]:
df = pd.read_csv(path, error_bad_lines=False)

In [4]:
df

,Tag,Significati
0,fridaysforfuture,"School Strike for Climate Swedish, also known ..."
1,climatestrike,"School Strike for Climate Swedish, also known ..."
2,climateaction,Climate action (or climate change action) refe...
3,climatecrisis,Climate crisis is a term describing global war...
4,climatechange,Contemporary climate change includes both glob...
...,...,...
312,krieg,Als Krieg wird ein organisierter und unter Ein...
313,astronaut,Ein Raumfahrer (auch Weltraumfahrer) ist ein T...
314,maurer,"Ein Maurer ist ein Bauhandwerker, dessen namen..."
315,wattenmeer,Wattenmeere (abgeleitet von Watt) sind bestimm...


In [5]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [6]:
tags = df['Tag']
meaning = df['Significati']

In [7]:
def clean_text(corpus):
    clear_text = []
    for phrase in corpus:
        # Replace URL with the tag url
        phrase = re.sub("https*\S+", "url", phrase)
        # Remove strange characters
        # &quot => ""
        # &#39 => '
        # &amp; => &
        # &lt; => <
        # &gt; => >
        phrase = re.sub("&quot;|\r|\n\|&#39|&amp;|&lt;|&gt;", "", phrase)

        # Punctuaction removal
        phrase = re.sub('[%s]' % re.escape(string.punctuation), ' ', phrase)
        # Replace numbers with the tag number
        phrase = re.sub("\d+", "number ", phrase)
        # Convert to lowercase
        phrase = phrase.lower()
        # Replace the over spaces
        phrase = re.sub('\s{2,}', " ", phrase)
        clear_text.append([phrase])
    return clear_text

In [8]:
clean_text

<function __main__.clean_text(corpus)>

In [9]:
cleared_text = clean_text(meaning)

In [10]:
cleared_text

[['school strike for climate swedish also known variously as fridays for future fff youth for climate climate strike or youth strike for climate is an international movement of school students who skip friday classes to participate in demonstrations to demand action from political leaders to prevent climate change and for the fossil fuel industry to transition to renewable energy publicity and widespread organising began after swedish pupil greta thunberg staged a protest in august number outside of the swedish riksdag parliament holding a sign that read skolstrejk för klimatet school strike for climate a global strike on number march number gathered more than one million strikers in number number strikes organised in number countries on number may number in the second glo'],
 ['school strike for climate swedish also known variously as fridays for future fff youth for climate climate strike or youth strike for climate is an international movement of school students who skip friday clas

In [11]:
import networkx as nx
import itertools
from operator import itemgetter

In [12]:
G = nx.read_graphml("../Dataset FFF/Graph_data/Real_Network.graphml")

In [13]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)

In [14]:
sample_nodes_1057 = []
for i in sorted_degree:
    if i[1] > 30:
        sample_nodes_1057.append(i[0])
print(len(sample_nodes_1057))

511


In [15]:
def select_sample_meaning(sample, meaning):
    sampled_meaning = []
    sample_nodes = []
    for i in sample:
        for j in range(len(tags)):
            if tags[j]==i:
                sampled_meaning.append(i + " " + meaning[j][0])
                sample_nodes.append(i)
    return sampled_meaning, sample_nodes

In [16]:
sampled_meaning_1057, sample_nodes_1057 = select_sample_meaning(sample_nodes_1057, cleared_text)

In [17]:
"""
for i in range(len(sample_nodes_1057)):
    if not isNaN(tags[i]):
        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]
    else:
        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]
"""

'\nfor i in range(len(sample_nodes_1057)):\n    if not isNaN(tags[i]):\n        sampled_meaning_1057[i] = sample_nodes_1057[i] + " " + sampled_meaning_1057[i][0]\n    else:\n        sampled_meaning_1057[i] = "nan " + sampled_meaning_1057[i][0]\n'

In [18]:
word_tokens = []
for phrase in sampled_meaning_1057:
    word_tokens.append(phrase.split())
stop_words = stopwords.words('english')
no_sw_1057=[]
for phrase in word_tokens:
    no_sw_tmp=[]
    for word in phrase:
        if word not in stop_words:
            no_sw_tmp.append(word)
    no_sw_1057.append(no_sw_tmp)

In [19]:
cleared_meaning_1057 = []
for i in no_sw_1057:
    new_s = ""
    for j in i:
        new_s = new_s + " " + j
    cleared_meaning_1057.append(new_s)

In [20]:
df = pd.DataFrame({"Tags": sample_nodes_1057, "Meaning" : cleared_meaning_1057})
df.to_csv("nodes_1057_meaning.csv", index=False)

In [21]:
from gensim.models import Word2Vec

In [22]:
def check_isin(word2vec, data):
    not_in = []
    for i in data:
        try:
            word2vec.wv[i]
        except:
            not_in.append(i)
    print(len(not_in))

In [23]:
word2vec_1057 = Word2Vec(no_sw_1057, min_count=1)

In [24]:
len(word2vec_1057.wv)

2810

# 2 - Store word2vec embedding

In [25]:
from linkpred.evaluation import Pair
import pickle

In [26]:
H_1057 = G.subgraph(sample_nodes_1057)

In [27]:
first = []
second = []
for i in H_1057.edges():
    first.append(word2vec_1057.wv[i[0]])
    second.append(word2vec_1057.wv[i[1]])
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("embedding_pickle_1057.csv")

In [28]:
word2vec_1057.save("word2vec_1057.model")

In [29]:
nodes = list(H_1057.nodes())
universe_1057 = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

In [30]:
with open('universe_1057.pickle', 'wb') as f:
    pickle.dump(universe_1057, f)

In [31]:
first = []
second = []
for i in H_1057.edges():
    first.append(i[0])
    second.append(i[1])

In [32]:
new_df = pd.DataFrame({"first":first, "second":second})
new_df.reset_index(inplace=True, drop=True)
new_df.to_pickle("edges_original_1057.csv")

In [33]:
nx.write_graphml(H_1057,"H_1057.graphml")

-------------------------------------